### Inference from a list of echo studies ###

In [1]:
import os
import glob
import pickle
import pandas as pd

# Custom imports
from werdich_cfr.tfutils.Modeltrainer import VideoTrainer

TensorFlow Version: 2.2.0


In [2]:
def read_model_dict(file):
    with open(file, 'rb') as f:
        model_dict = pickle.load(f)
    return model_dict

# Directories and data sets
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
meta_date = '200606'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+meta_date)
file_df_file = 'echo_BWH_npy_feather_files_'+meta_date+'.parquet'
view = 'a4c'

# echo list
echo_list_dir = os.path.normpath('/mnt/obi0/andreas/data/cfr/predictions_echodata/SecondEchoGenetics')
echo_list_filename = 'a4cname_BWH.txt'
echo_list_file = os.path.join(echo_list_dir, echo_list_filename)
echo_list = pd.read_csv(echo_list_file, header='infer', sep='\t')
echo_file_list = list(echo_list.names.unique())
print(f'Original file list: {len(echo_file_list)} files.')

best_models = pd.read_parquet(os.path.join(echo_list_dir, 'cfr_models_200611.parquet')).reset_index(drop=True)

Original file list: 26537 files.


In [3]:
echo_list.head().names.iloc[0]

'4f2101e60be2be_4903a44ab09602c1f5c8e634489d_Image-45.npy.lz4'

In [3]:
# Create a reduced npy_feather files to speed up the metadata collections

#file_df = pd.read_parquet(os.path.join(meta_dir, file_df_file))
#file_df_study = file_df[file_df.study.isin(study_list)]
#file_df_study_list = list(file_df_study.study.unique())
#print(len(study_list))
#print(len(file_df_study_list))
# What's missing?
#missing_files = list(set(study_list).difference(file_df_study_list))
#print(len(missing_files))

# Save missing echo list
#echo_list_file_missing = echo_list_filename.split('.')[0]+'_missing.parquet'
#echo_list_missing = echo_list[echo_list.study.isin(missing_files)]
#echo_list_missing.to_parquet(os.path.join(meta_dir, echo_list_file_missing))

# Save the new list to collect the metadata
#file_df_file_pred = os.path.join(meta_dir, 'echo_BWH_npy_feather_files_pred_'+meta_date+'.parquet')
#file_df_study.to_parquet(file_df_file_pred)

7860
7705
155


In [3]:
# Join the echo list with the meta data and filter VIEWS!!!
meta_file = os.path.join(cfr_data_root, 'metadata_200606', 'echo_BWH_meta_200606.parquet')
meta_df = pd.read_parquet(meta_file)
print(meta_df.shape)

(2257427, 42)


In [4]:
# Filter the meta data by the echos we need
meta_df = meta_df[meta_df.filename.isin(echo_file_list)]
print(f'Number of files in meta data after echo filter: {len(meta_df.filename.unique())}')
print(meta_df.shape)

# Remove meta rows without view classification
meta_df = meta_df.loc[~meta_df.a4c.isnull()]

# Remove rows without frame_time
meta_df = meta_df.loc[~meta_df.frame_time.isnull()].reset_index(drop = True)

print()
print('After removal of rows without view clasification:')
print('Total number of patients in meta data {}'.format(len(meta_df.mrn.unique())))
print('Total number of studies {}'.format(len(meta_df.study.unique())))
print('Total number of files in meta data {}'.format(len(meta_df.filename.unique())))

Number of files in meta data after echo filter: 26537
(26537, 42)

After removal of rows without view clasification:
Total number of patients in meta data 5945
Total number of studies 9415
Total number of files in meta data 26537


In [5]:
view_dict = {'view_a2c': ['a2c', 'a2c_laocc', 'a2c_lvocc_s'],
             'view_a3c': ['a3c', 'a3c_laocc', 'a3c_lvocc_s'],
             'view_a4c': ['a4c', 'a4c_far', 'a4c_laocc', 'a4c_lvocc_s', 'a4c_rv', 'a4c_rv_laocc'],
             'view_plax': ['plax_far', 'plax_lac', 'plax_laz', 'plax_laz_ao', 'plax_plax'],
             'view_psax': ['psax_avz', 'psax_az', 'psax_mv', 'psax_pap'],
             'view_other': ['other', 'a5c', 'apex', 'rvinf', 'subcostal', 'suprasternal']}

# Get the maxiumum view classification score for each row
view_list = [item for sublist in view_dict.values() for item in sublist]
echo_meta_sum_views = meta_df.assign(max_view = meta_df[view_list].\
                                     idxmax(axis = 1))

# Make sure that we have all views in our list and for consistency: Sum up all view columns
echo_meta_sum_views = echo_meta_sum_views.assign(sum_views = echo_meta_sum_views[view_list].sum(axis=1))

print(f'Studies in data:     {len(echo_meta_sum_views.study.unique())}')
print(f'Total files:         {len(echo_meta_sum_views.filename.unique())}')

meta_df_a4c = echo_meta_sum_views[echo_meta_sum_views.max_view==view].reset_index(drop=True)

print()
print(f'Studies in a4c data: {len(meta_df_a4c.study.unique())}')
print(f'Total files:         {len(meta_df_a4c.filename.unique())}')
print(f'Total patients:      {len(meta_df_a4c.mrn.unique())}')

Studies in data:     9415
Total files:         26537

Studies in a4c data: 9415
Total files:         26537
Total patients:      5945


In [11]:
meta_a4c_filename = echo_list_filename.split('.')[0] + '_'+view+'.parquet'
print(meta_a4c_filename)

a4cname_MGH_a4c.parquet


In [12]:
meta_a4c_filename = echo_list_filename.split('.')[0] + '_'+view+'.parquet'
meta_a4c_file = os.path.join(echo_list_dir, meta_a4c_filename)
meta_df_a4c.to_parquet(meta_a4c_file)
meta_df_a4c.head(2)

,filename,dir,study,mrn,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,sum_views
0,48b090129a7ecc_4903a582ec7227e72ca1a12db852_Im...,/mnt/obi0/phi/echo/npyFiles/MGH/48b0/48b090129...,48b090129a7ecc_4903a582ec7227e72ca1a12db852,3517085,2017-03-29 10:52:45,48b090129a7ecc_4903a582ec7227e72ca1a12db852_Im...,MGH,EPIQ 7C,Philips Medical Systems,0.0,33.333,76.0,48.0,0.038599,0.038599,4.369181e-14,1.002213e-11,4.952000e-17,9.540701e-20,8.480631e-19,3.362204e-15,1.000000,1.380723e-14,1.235565e-16,1.717246e-15,1.413478e-16,1.533292e-16,2.999292e-17,3.079603e-17,2.827710e-22,6.419594e-17,4.401530e-16,2.819618e-16,3.035525e-18,6.578000e-17,9.705355e-16,1.937740e-14,8.963561e-17,1.988039e-15,1.589031e-20,2.990445e-20,2.887505e-18,a4c,1.0
1,48b09016b32e8b_4903a580509311eb732873606d54_Im...,/mnt/obi0/phi/echo/npyFiles/MGH/48b0/48b09016b...,48b09016b32e8b_4903a580509311eb732873606d54,3513257,2015-04-14 08:24:23,48b09016b32e8b_4903a580509311eb732873606d54_Im...,MGH,iE33,Philips Medical Systems,0.0,33.333,91.0,42.0,0.047111,0.047111,3.749047e-07,2.990790e-10,2.754385e-09,4.811635e-08,3.790895e-10,8.239009e-09,0.996496,1.198259e-08,1.066312e-10,8.882178e-09,1.425669e-03,5.240650e-09,1.404470e-08,1.202118e-09,6.229320e-08,1.117606e-08,4.633523e-11,1.920444e-09,4.513532e-11,1.979736e-09,3.215599e-09,8.065429e-06,1.350255e-10,2.069227e-03,1.685130e-08,9.140099e-11,4.707263e-11,a4c,1.0


In [14]:
meta_df_a4c.shape

(13273, 44)

In [13]:
model = best_models
model

,model_name,model_output,epoch,chechkpoint_file,spear_cor,spear_p,pear_cor,pear_p,n_samples,dset,view,gpu
0,nondefect_a4c_dgx-1_rest_mbf_unaff,rest_mbf_unaff,100,/mnt/obi0/andreas/data/cfr/log/nondefect_a4c_d...,0.343776,7.655502e-26,0.350960,6.191784e-27,881,nondefect,a4c,dgx-1
1,nondefect_a4c_dgx-1_stress_mbf_unaff,stress_mbf_unaff,100,/mnt/obi0/andreas/data/cfr/log/nondefect_a4c_d...,0.446963,1.734637e-44,0.440022,5.112244e-43,881,nondefect,a4c,dgx-1
2,nondefect_a4c_dgx-1_unaffected_cfr,unaffected_cfr,100,/mnt/obi0/andreas/data/cfr/log/nondefect_a4c_d...,0.226273,1.083594e-11,0.263326,1.934262e-15,881,nondefect,a4c,dgx-1
3,global_a4c_gpu2_stress_global_mbf,stress_global_mbf,150,/mnt/obi0/andreas/data/cfr/log/global_a4c_gpu2...,0.521974,1.406801e-74,0.532282,5.110090e-78,1052,global,a4c,gpu2
4,global_a4c_dgx-1_rest_global_mbf,rest_global_mbf,150,/mnt/obi0/andreas/data/cfr/log/global_a4c_dgx-...,0.444414,3.753792e-52,0.458937,6.343615e-56,1052,global,a4c,dgx-1
5,global_a4c_gpu2_global_cfr_calc,global_cfr_calc,50,/mnt/obi0/andreas/data/cfr/log/global_a4c_gpu2...,0.325327,2.339397e-27,0.354585,1.593943e-32,1052,global,a4c,gpu2


In [20]:
# Load disqualified files after predictions
dis_df_file_BWH = 'a4cname_BWH_a4c_disqualified.parquet'
echo_df_file_BWH = 'a4cname_BWH_a4c.parquet'
echo_list_filename = 'a4cname_BWH.txt'
echo_list_file = os.path.join(echo_list_dir, echo_list_filename)
echo_list = pd.read_csv(echo_list_file, header='infer', sep='\t')
echo_file_list_BWH = list(echo_list.names.unique())
dis_df_BWH = pd.read_parquet(os.path.join(echo_list_dir, dis_df_file_BWH))
echo_df_BWH = pd.read_parquet(os.path.join(echo_list_dir, echo_df_file_BWH))
print('BWH')
print(f'Files in original list:       {len(echo_file_list_BWH)}')
print(f'Files with .npy and metadata: {len(echo_df_BWH.filename.unique())}')
print(f'Disqualified files:           {len(dis_df_BWH.filename.unique())}')


dis_df_file_MGH = 'a4cname_MGH_a4c_disqualified.parquet'
echo_df_file_MGH = 'a4cname_MGH_a4c.parquet'
echo_list_filename = 'a4cname_MGH.txt'
echo_list_file = os.path.join(echo_list_dir, echo_list_filename)
echo_list = pd.read_csv(echo_list_file, header='infer', sep='\t')
echo_file_list_MGH = list(echo_list.names.unique())

dis_df_MGH = pd.read_parquet(os.path.join(echo_list_dir, dis_df_file_MGH))
echo_df_MGH = pd.read_parquet(os.path.join(echo_list_dir, echo_df_file_MGH))

print()
print('MGH')
print(f'Files in original list:       {len(echo_file_list_MGH)}')
print(f'Files with .npy and metadata: {len(echo_df_MGH.filename.unique())}')
print(f'Disqualified files:           {len(dis_df_MGH.filename.unique())}')

BWH
Files in original list:       26537
Files with .npy and metadata: 26537
Disqualified files:           4869

MGH
Files in original list:       13273
Files with .npy and metadata: 13273
Disqualified files:           4309


In [61]:
4309/13273*100

32.46440141640925

In [62]:
max_frame_time=33.34 ### FINAL CUTOFF FOR FRAME TIME INTERVAL
dis_df_BWH = dis_df_BWH.assign(videolen=(dis_df_BWH.frame_time * 1e-3 * dis_df_BWH.number_of_frames))
min_len = max_frame_time*40

dis_df_BWH_filter = dis_df_BWH[(dis_df_BWH.frame_time<=max_frame_time*1000) & (dis_df_BWH.videolen>=min_len)]

In [63]:
dis_df_BWH_filter

,filename,dir,study,mrn,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,sum_views,videolen
